# Tax Appeal
### Created by: Ryan Jinnette

#### This file uses selenium and pandas to scrape county website for tax records that will be used for analysis to fair price valuation

## Set Up and Imports

In [56]:
# !pip install selenium
# !pip install pandas
# !pip install rpy2

import pandas as pd
from selenium.webdriver.common.by import By
from selenium import webdriver
import chromedriver_autoinstaller
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm
import rpy2
%load_ext rpy2.ipython

#import timeit

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


## Constants and Class Definitions

In [57]:
class lot:
    '''This class provides a framework to store data more easily instead of multiple lists.
    I can reference just the instance of the class'''
    def __init__(self, prop_id=None, geo_id=None, name=None, appraised=None, land_value=None, land_size=None, improved_value=None, sqft=None, year=None, bldg_class=None):
        self.prop_id = prop_id
        self.geo_id = geo_id
        self.name = name
        self.appraised = appraised
        self.land_value = land_value
        self.land_size = land_size
        self.improved_value = improved_value
        self.imp_sqft = sqft
        self.year = year
        self.bldg_class = bldg_class
        self.dollar_sqft = None
    
    def set_dtypes(self):
        '''used to keep things consistent so that math can be done on all cells and not throw errors'''
        self.improved_value = float(self.improved_value.replace('(+)','').replace(',','').replace('$',''))
        self.appraised = float(self.appraised.replace(',','').replace('$',''))
        self.land_value = float(self.land_value.replace(',','').replace('$',''))
        self.land_size = float(self.land_size.replace(',',''))
        if self.imp_sqft != 0:
            self.imp_sqft = float(self.imp_sqft[13:-4].strip().replace(',',''))
            self.dollar_sqft = round(self.improved_value / self.imp_sqft,2)
        
            


In [58]:
lots = []
web_url = r'https://esearch.galvestoncad.org/Search/Result?keywords=Johnson%20Crawford#'


## Data Collection
#### Iterates over the webpage and grabs all necessary data, adds to the class and uses the XPATH of certain items to track them on the page for easy identification and repeatability

In [59]:
def collect(web_url):
    #options = webdriver.ChromeOptions()
    #options.add_argument('--headless')
    #driver = webdriver.Chrome(options = options)
    driver = webdriver.Chrome()
    driver.get(web_url)
    driver.implicitly_wait(2) # dynamic waiting to see when any certain searched for object becomes available
    time.sleep(2)
    
    #change the list view to 100 instead of default 25
    btn = driver.find_element(By.ID, 'btnFilterByPage')
    btn.click()
    
    btn = driver.find_element(By.XPATH, '/html/body/div[4]/div[3]/div/span[3]/ul/li[3]')
    btn.click()
    
    time.sleep(2)

    rows = driver.find_elements(By.XPATH, '/html/body/div[4]/div[4]/div[2]/table/tbody/tr')
    start = time.time()
    
    for i in range(2, len(rows)//10):
        
        
        # this grabs all initially avail info, will have to go into each property next
        prop_id = driver.find_element(By.XPATH, f'/html/body/div[4]/div[4]/div[2]/table/tbody/tr[{i}]/td[1]').text
        geo_id = driver.find_element(By.XPATH, f'/html/body/div[4]/div[4]/div[2]/table/tbody/tr[{i}]/td[3]').text
        name = driver.find_element(By.XPATH, f'/html/body/div[4]/div[4]/div[2]/table/tbody/tr[{i}]/td[6]').text
        appraised = driver.find_element(By.XPATH, f'/html/body/div[4]/div[4]/div[2]/table/tbody/tr[{i}]/td[10]').text
        
        #go into each property and get more details
        click_property = driver.find_element(By.XPATH, f'/html/body/div[4]/div[4]/div[2]/table/tbody/tr[{i}]')
        click_property.click()
        
        land_size = driver.find_element(By.XPATH, f'/html/body/div[4]/div[1]/div[5]/div[2]/table/tbody/tr[2]/td[4]').text
        land_value = driver.find_element(By.XPATH, f'/html/body/div[4]/div[1]/div[5]/div[2]/table/tbody/tr[2]/td[7]').text
        
        # in case any of the following don't exist. (I.E. the lot is land only)
        try:
            improvement = driver.find_element(By.XPATH, f'/html/body/div[4]/div[1]/div[2]/div[2]/div/table/tbody/tr[3]/td').text
        except NoSuchElementException:
            improvement = 0
        if improvement != '$0 (+)':
            try:
                imp_sqft = driver.find_element(By.XPATH, f'/html/body/div[4]/div[1]/div[4]/div[2]/div[1]/span[3]').text
                if 'State' in imp_sqft: #some of the pages are not consistent formatting, handles that
                    imp_sqft = driver.find_element(By.XPATH, f'/html/body/div[4]/div[1]/div[4]/div[2]/div[1]/span[4]').text
            except NoSuchElementException:
                imp_sqft = 0

            try:
                year = driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[4]/div[2]/table/tbody/tr[2]/td[5]').text
            except NoSuchElementException:
                year = 'Nan'

            try:
                bldg_class = driver.find_element(By.XPATH, f'/html/body/div[4]/div[1]/div[4]/div[2]/table/tbody/tr[2]/td[3]').text
            except NoSuchElementException:
                bldg_class = 'Nan'
        else:
            imp_sqft = 0
            year = 'Nan'
            bldg_class = 'Nan'

        
        lot_info = lot(prop_id, geo_id, name, appraised, land_value,land_size, improvement, imp_sqft, year, bldg_class)
        lots.append(lot_info)
        driver.back()
    finish = time.time()
    print(f"took {round(finish-start,2)} seconds")
        
collect(web_url)




took 5.11 seconds


### Clean Data and Set Dtypes


In [60]:
for lot in lots:
    lot.set_dtypes()
    print(lot.imp_sqft, type(lot.imp_sqft))

0 <class 'int'>
2577.0 <class 'float'>
1564.0 <class 'float'>
2209.0 <class 'float'>


### Converting gathered data into csv export

In [61]:
def convert_to_csv():
    attribute_names = list(vars(lots[0]).keys())
    data = [{attr: getattr(lot, attr) for attr in attribute_names} for lot in lots]
    df = pd.DataFrame(data)
    df.to_csv('data.csv', index = False)

convert_to_csv()

In [62]:
%%R
#install.packages("ggplot2")
#install.packages("tidyverse")
library(ggplot2)
library(tidyverse)

In [63]:
%%R
new_df = read.csv('data.csv')
head(new_df)

  prop_id             geo_id                       name appraised land_value
1  221853 4275-0000-0001-000 CASON GREGORY A & JONICA A    289040     289040
2  221854 4275-0000-0003-000 CASON GREGORY A & JONICA A    770000     163500
3  221855 4275-0000-0004-000  MORRIS GARRETT & JENNIFER    535520     161660
4  221856 4275-0000-0005-000      COLLINS WILLIAM DAVID    769960     161660
  land_size improved_value imp_sqft year bldg_class dollar_sqft
1     17986              0        0  NaN        Nan          NA
2      8175         606500     2577 2016       BH14      235.35
3      8083         373860     1564 2014       BH12      239.04
4      8083         608300     2209 2014       BH14      275.37
